## 랩 1: 간단한 고객 지원 에이전트 프로토타입 생성

### 개요

[Amazon Bedrock AgentCore](https://aws.amazon.com/bedrock/agentcore/)는 모든 프레임워크와 모델을 사용하여 AI 에이전트를 안전하게 대규모로 배포하고 운영할 수 있도록 도와줍니다. 프로토타입에서 프로덕션으로 더 빠르게 이동할 수 있는 기능을 제공합니다.

이 5개 랩 튜토리얼에서는 **고객 지원 에이전트**를 사용하여 프로토타입에서 프로덕션까지의 엔드투엔드 여정을 보여드리겠습니다. 이 예제에서는 에이전트 구축을 위한 간단하고 코드 우선 프레임워크인 [Strands Agents](https://strandsagents.com/latest/)와 Amazon Bedrock의 Anthropic Claude Sonnet 3.7 모델을 사용합니다. 애플리케이션에서는 원하는 프레임워크와 모델을 사용할 수 있습니다. 여기서 다루는 개념들은 다른 프레임워크와 모델에서도 적용할 수 있다는 점이 중요합니다.

**워크샵 여정:**
- **랩 1 (현재)**: 에이전트 프로토타입 생성 - 기능적인 고객 지원 에이전트 구축
- **랩 2**: 메모리로 향상 - 대화 컨텍스트와 개인화 추가
- **랩 3**: 게이트웨이 및 아이덴티티로 확장 - 에이전트 간 도구 안전하게 공유
- **랩 4**: 프로덕션 배포 - 관찰 가능성을 갖춘 AgentCore 런타임 사용
- **랩 5**: 사용자 인터페이스 구축 - 고객 대면 애플리케이션 생성

이 첫 번째 랩에서는 워크샵 전반에 걸쳐 지속적인 메모리, 공유 도구, 완전한 관찰 가능성을 갖춘 여러 고객을 서비스하는 프로덕션 준비 시스템으로 발전할 고객 지원 에이전트 프로토타입을 구축합니다. 우리 에이전트는 다음과 같은 로컬 도구를 사용할 수 있습니다:
- **get_return_policy()** - 특정 제품의 반품 정책 가져오기
- **get_product_info()** - 제품 정보 가져오기
- **web_search()** - 문제 해결 도움을 위한 웹 검색


### 랩 1 아키텍처
<div style="text-align:left">
    <img src="images/architecture_lab1_strands.png" width="75%"/>
</div>

*로컬에서 실행되는 간단한 프로토타입. 후속 랩에서는 이를 공유 도구, 지속적인 메모리, 프로덕션급 관찰 가능성을 갖춘 AgentCore 서비스로 마이그레이션합니다.*

### 전제조건

* 적절한 권한을 가진 **AWS 계정**
* 로컬에 설치된 **Python 3.10+**
* 자격 증명으로 구성된 **AWS CLI**
* [Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)에서 활성화된 **Anthropic Claude 3.7**
* 다음 셀에서 설치될 **Strands Agents** 및 기타 라이브러리

### 1단계: 종속성 설치 및 라이브러리 가져오기
시작하기 전에 이 랩에 필요한 전제조건을 설치해보겠습니다

In [ ]:
# Install required packages
%pip install -U -r requirements.txt -q

이제 필요한 라이브러리를 가져오고 boto3 세션을 초기화할 수 있습니다

In [ ]:
# Import libraries
import boto3
from boto3.session import Session

from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

from strands.tools import tool
from scripts.utils import put_ssm_parameter

In [ ]:
# Get boto session
boto_session = Session()
region = boto_session.region_name

### 2단계: 사용자 정의 도구 구현

다음으로 고객 지원 에이전트에 제공될 3개의 도구를 구현하겠습니다.

Strands Agent에서 도구를 정의하는 것은 매우 간단합니다. 함수에 `@도구` 데코레이터를 추가하고 함수의 docstring에 도구 설명을 제공하기만 하면 됩니다. Strands Agents는 함수 문서, 타입 및 인수를 사용하여 에이전트에 이 도구에 대한 컨텍스트를 제공합니다.


#### 도구 1: 반품 정책 가져오기

**목적:** 이 도구는 고객이 다양한 제품 카테고리의 반품 정책을 이해할 수 있도록 도와줍니다. 반품 기간, 조건, 절차, 환불 일정에 대한 자세한 정보를 제공하여 고객이 제품을 반품할 때 무엇을 기대할 수 있는지 정확히 알 수 있도록 합니다.

In [ ]:
@tool
def get_return_policy(product_category: str) -> str:
    """
    특정 제품 카테고리에 대한 반품 정책 정보를 가져옵니다.

    Args:
        product_category: 전자제품 카테고리 (예: 'smartphones', 'laptops', 'accessories')

    Returns:
        기간과 조건을 포함한 서식화된 반품 정책 세부사항
    """
    # 모의 반품 정책 데이터베이스 - 실제 구현에서는 정책 DB를 조회함
    return_policies = {
        "smartphones": {
            "window": "30일",
            "condition": "원래 포장, 물리적 손상 없음, 공장 초기화 필요",
            "process": "온라인 RMA 포털 또는 기술 지원",
            "refund_time": "검수 후 영업일 기준 5~7일",
            "shipping": "무료 반품 배송, 선불 라벨 제공",
            "warranty": "1년 제조사 보증 포함"
        },
         "laptops": {
            "window": "30일", 
            "condition": "원래 포장, 모든 액세서리 포함, 소프트웨어 수정 없음",
            "process": "반품 전 기술 지원 검증 필요",
            "refund_time": "검수 후 영업일 기준 7~10일",
            "shipping": "원래 포장과 함께 무료 반품 배송",
            "warranty": "1년 제조사 보증, 연장 옵션 가능"
        },
        "accessories": {
            "window": "30일",
            "condition": "개봉하지 않은 포장 권장, 모든 구성품 포함",
            "process": "온라인 반품 포털",
            "refund_time": "수령 후 영업일 기준 3~5일",
            "shipping": "5만 원 이하 제품은 고객이 반품 배송비 부담",
            "warranty": "90일 제조사 보증"
        }
    }

    # 나열되지 않은 카테고리의 기본 정책
    default_policy = {
        "window": "30일",
        "condition": "모든 구성품이 포함된 원래 상태",
        "process": "기술 지원팀에 문의",
        "refund_time": "검수 후 영업일 기준 5~7일", 
        "shipping": "반품 배송 정책은 카테고리별로 상이함",
        "warranty": "제조사 표준 보증 적용"
    }

    policy = return_policies.get(product_category.lower(), default_policy)
    return f"📦 반품 정책 - {product_category.title()}:\n\n" \
           f"• 반품 가능 기간: 배송일로부터 {policy['window']}\n" \
           f"• 조건: {policy['condition']}\n" \
           f"• 절차: {policy['process']}\n" \
           f"• 환불 소요 기간: {policy['refund_time']}\n" \
           f"• 배송: {policy['shipping']}\n" \
           f"• 보증: {policy['warranty']}"

print("✅ Return policy tool ready")

#### 도구 2: 제품 정보 가져오기

**목적:** 이 도구는 보증, 사용 가능한 모델, 주요 기능, 배송 정책, 반품 정보를 포함한 포괄적인 제품 세부 사항을 고객에게 제공합니다. 고객이 정보에 기반한 구매 결정을 내리고 구매하는 제품을 이해할 수 있도록 도와줍니다.

In [ ]:
@tool
def get_product_info(product_type: str) -> str:
    """
    전자제품의 상세 기술 사양 및 정보를 가져옵니다.

    Args:
        product_type: 전자제품 유형 (예: 'laptops', 'smartphones', 'headphones', 'monitors')
    Returns:
        보증, 기능, 정책을 포함한 서식화된 제품 정보
    """
    # 모의 제품 카탈로그 - 실제 구현에서는 제품 데이터베이스를 조회함
    products = {
        "laptops": {
            "warranty": "1년 제조사 보증 + 선택적 연장 보증",
            "specs": "Intel/AMD 프로세서, 8-32GB RAM, SSD 저장장치, 다양한 디스플레이 크기",
            "features": "백라이트 키보드, USB-C/Thunderbolt, Wi-Fi 6, Bluetooth 5.0",
            "compatibility": "Windows 11, macOS, Linux (모델별 상이)",
            "support": "기술 지원 및 드라이버 업데이트 제공"
        },
        "smartphones": {
            "warranty": "1년 제조사 보증",
            "specs": "5G/4G 연결, 128GB-1TB 저장공간, 다중 카메라 시스템",
            "features": "무선 충전, 방수 기능, 생체 인식 보안",
            "compatibility": "iOS/Android, 통신사 언락 옵션 가능",
            "support": "소프트웨어 업데이트 및 기술 지원 제공"
        },
        "headphones": {
            "warranty": "1년 제조사 보증",
            "specs": "유선/무선 옵션, 노이즈 캔슬링, 20Hz-20kHz 주파수",
            "features": "액티브 노이즈 캔슬링, 터치 컨트롤, 음성 어시스턴트 지원",
            "compatibility": "Bluetooth 5.0+, 3.5mm 단자, USB-C 충전",
            "support": "전용 앱을 통한 펌웨어 업데이트"
        },
        "monitors": {
            "warranty": "3년 제조사 보증",
            "specs": "4K/1440p/1080p 해상도, IPS/OLED 패널, 다양한 크기",
            "features": "HDR 지원, 고주사율, 높이 조절 스탠드",
            "compatibility": "HDMI, DisplayPort, USB-C 입력",
            "support": "색상 보정 및 기술 지원"
        }
    }
    product = products.get(product_type.lower())
    if not product:
        return f"{product_type}에 대한 기술 사양을 확인할 수 없습니다. 상세 제품 정보와 호환성 요구사항은 기술 지원팀에 문의해 주세요."

    return f"📘 제품 기술 정보 - {product_type.title()}:\n\n" \
           f"• 보증: {product['warranty']}\n" \
           f"• 사양: {product['specs']}\n" \
           f"• 주요 기능: {product['features']}\n" \
           f"• 호환성: {product['compatibility']}\n" \
           f"• 지원: {product['support']}"

print("✅ get_product_info tool ready")

#### 도구 3: 웹 검색

**목적:** 이 도구를 통해 고객은 문제 해결 지원이나 제품 추천 등에 대한 제안을 받을 수 있습니다.

In [ ]:
@tool
def web_search(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """최신 정보를 웹에서 검색합니다.
    
    Args:
        keywords (str): 검색할 키워드.
        region (str): 검색 지역: wt-wt, us-en, uk-en, ru-ru 등.
        max_results (int | None): 반환할 최대 결과 수.
    Returns:
        검색 결과가 포함된 딕셔너리 리스트.
    
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "검색 결과가 없습니다."
    except RatelimitException:
        return "검색 제한에 도달했습니다. 잠시 후 다시 시도해 주세요."
    except DDGSException as e:
        return f"검색 오류: {e}"
    except Exception as e:
        return f"검색 오류: {str(e)}"

print("✅ Web search tool ready")

### 4단계: 고객 지원 에이전트 생성 및 구성

다음으로 모델, 이전 단계에서 구현한 도구 목록, 시스템 프롬프트를 제공하여 고객 지원 에이전트를 생성하겠습니다.

In [ ]:
from strands import Agent
from strands.models import BedrockModel

SYSTEM_PROMPT = """당신은 전자제품 이커머스 회사의 친절하고 전문적인 고객 지원 어시스턴트입니다.  
당신의 역할은 다음과 같습니다:
- 사용 가능한 도구를 활용해 정확한 정보를 제공합니다.
- 고객에게 기술 정보와 제품 사양을 지원합니다.
- 친절하고, 인내심 있으며, 고객을 이해하는 태도로 응대합니다.
- 질문에 답변한 후 항상 추가적인 도움이 필요한지 제안합니다.
- 만약 도움을 줄 수 없는 경우, 적절한 담당 부서나 연락처를 안내합니다.

다음과 같은 도구에 접근할 수 있습니다:
1. get_return_policy() - 보증 및 반품 정책 문의 시 사용
2. get_product_info() - 특정 제품에 대한 정보 확인 시 사용
3. web_search() - 최신 기술 문서나 업데이트된 정보를 확인할 때 사용

전자제품이나 사양에 대해 추측하지 말고, 항상 적절한 도구를 사용해 정확하고 최신의 정보를 제공해야 합니다.
"""

# Initialize the Bedrock model (Anthropic Claude 3.7 Sonnet)
model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    temperature=0.3,
    region_name=region
)

# Create the customer support agent with all tools
agent = Agent(
    model=model,
    tools=[
        get_product_info,  # Tool 1: Simple product information lookup
        get_return_policy,  # Tool 2: Simple return policy lookup
        web_search # Tool 3: Access the web for updated information
    ],
    system_prompt=SYSTEM_PROMPT,
)

print("Customer Support Agent created successfully!")

### 5단계: 고객 지원 에이전트 테스트

모든 도구가 올바르게 작동하는지 확인하기 위해 샘플 쿼리로 에이전트를 테스트해보겠습니다.

#### 반품 확인 테스트

In [ ]:
response = agent("갤럭시탭의 반품 정책은 무엇입니까?")

#### 문제 해결 테스트

In [ ]:
response = agent("지난달에 갤럭시 폴드를 샀는데, 발열이 심해서 마음에 안 들어요. 어떻게 해결해야 할까요?")

## 🎉 랩 1 완료!

기능적인 고객 지원 에이전트 프로토타입을 성공적으로 생성했습니다! 달성한 내용은 다음과 같습니다:

- 3개의 사용자 정의 도구(반품 정책, 제품 정보, 웹 검색)를 갖춘 에이전트 구축
- 다중 도구 상호작용 및 웹 검색 기능 테스트
- 프로덕션 여정을 위한 기초 구축

### 현재 한계 (이것들을 해결할 예정입니다!)
- **단일 사용자 대화 메모리** - 로컬 대화 세션, 여러 고객에게는 여러 세션이 필요
- **세션으로 제한된 대화 기록** - 대화에서 장기 메모리나 세션 간 정보를 사용할 수 없음
- **도구 재사용성** - 도구를 다른 에이전트에서 재사용할 수 없음
- **로컬에서만 실행** - 확장 불가능
- **아이덴티티** - 사용자 및/또는 에이전트 아이덴티티나 액세스 제어 없음
- **관찰 가능성** - 에이전트 동작에 대한 제한적인 관찰 가능성
- **기존 API** - 고객 데이터를 위한 기존 엔터프라이즈 API에 액세스 불가

##### 다음 단계 [랩 2: 메모리 추가로 에이전트 개인화 →](lab-02-agentcore-메모리.ipynb)
